In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv
import networkx as nx
import matplotlib.pyplot as plt

# ----------------------------
# 1. Cargar el dataset Cora
# ----------------------------
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]
print(f"Número de nodos: {data.num_nodes}, Número de aristas: {data.num_edges}")
print(f"Número de características por nodo: {data.num_node_features}")

# ----------------------------
# 2. Definir el modelo generador
# ----------------------------
class GraphGenerator(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GraphGenerator, self).__init__()
        self.encoder = GCNConv(in_channels, hidden_channels)  # Codificación
        self.decoder = torch.nn.Linear(hidden_channels, in_channels)  # Decodificación
        
    def forward(self, x, edge_index):
        # Codificación
        h = F.relu(self.encoder(x, edge_index))
        # Decodificación
        reconstructed_x = torch.sigmoid(self.decoder(h))
        return reconstructed_x, h

# ----------------------------
# 3. Configuración del modelo
# ----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphGenerator(in_channels=data.num_node_features, hidden_channels=32).to(device)  # Cambiar a 32 neuronas
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------------------------
# 4. Entrenamiento del modelo
# ----------------------------
def train():
    model.train()
    optimizer.zero_grad()
    reconstructed_x, _ = model(data.x, data.edge_index)
    loss = F.mse_loss(reconstructed_x, data.x)
    loss.backward()
    optimizer.step()
    return loss.item()

# Entrenamiento por 50 épocas
for epoch in range(50):
    loss = train()
    print(f"Época {epoch+1}, Pérdida: {loss:.4f}")

# ----------------------------
# 5. Generar un nuevo grafo
# ----------------------------
model.eval()
with torch.no_grad():
    _, embeddings = model(data.x, data.edge_index)

    # Generar nuevos bordes aleatoriamente basados en embeddings
    new_edges = []
    for i in range(data.num_nodes):
        for j in range(i + 1, data.num_nodes):
            prob = torch.sigmoid((embeddings[i] * embeddings[j]).sum()).item()
            if prob > 0.5:
                new_edges.append((i, j))

# Crear un nuevo grafo con nodos y bordes generados
new_graph = nx.Graph()
new_graph.add_edges_from(new_edges)

# ----------------------------
# 6. Visualizar el grafo original y el generado
# ----------------------------
# Grafo original
original_graph = to_networkx(data, to_undirected=True)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
nx.draw(original_graph, with_labels=False, node_color='lightblue', edge_color='gray', node_size=50)
plt.title("Grafo Original - Cora")

# Grafo generado
plt.subplot(1, 2, 2)
pos = nx.spring_layout(new_graph)
nx.draw(new_graph, pos, with_labels=False, node_color='lightgreen', edge_color='gray', node_size=50)
plt.title("Grafo Generado - Cora")
plt.show()


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Paso 1: Cargar el dataset PubMed
dataset = Planetoid(root='data/PubMed', name='PubMed')

# Mostrar detalles del dataset
print(f"Dataset: {dataset}:")
print(f"- Número de grafos: {len(dataset)}")
print(f"- Número de características por nodo: {dataset.num_node_features}")
print(f"- Número de clases: {dataset.num_classes}")

# Usar el primer grafo del dataset
data = dataset[0]

# Paso 2: Definir la GNN
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)  # De 500 -> 16
        self.conv2 = GCNConv(16, dataset.num_classes)       # De 16 -> 3

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Paso 3: Entrenar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Entrenamiento
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Paso 4: Evaluar el modelo
model.eval()
_, pred = model(data).max(dim=1)
correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
accuracy = correct / int(data.test_mask.sum())
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# Paso 5: Visualizar el grafo
import networkx as nx
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
G = to_networkx(data, to_undirected=True)
node_colors = pred.cpu().numpy()
nx.draw(G, node_size=50, node_color=node_colors, cmap='Set2')
plt.title("Clasificación de nodos en el grafo PubMed")
plt.show()


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Paso 1: Cargar el dataset CiteSeer
dataset = Planetoid(root='data/CiteSeer', name='CiteSeer')

# Mostrar detalles del dataset
print(f"Dataset: {dataset}:")
print(f"- Número de grafos: {len(dataset)}")
print(f"- Número de características por nodo: {dataset.num_node_features}")
print(f"- Número de clases: {dataset.num_classes}")

# Usar el primer grafo del dataset
data = dataset[0]

# Paso 2: Definir la GNN
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)  # De 3703 -> 16
        self.conv2 = GCNConv(16, dataset.num_classes)       # De 16 -> 6

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Paso 3: Entrenar el modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Entrenamiento
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Paso 4: Evaluar el modelo
model.eval()
_, pred = model(data).max(dim=1)
correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
accuracy = correct / int(data.test_mask.sum())
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# Paso 5: Visualizar el grafo
import networkx as nx
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
G = to_networkx(data, to_undirected=True)
node_colors = pred.cpu().numpy()
nx.draw(G, node_size=50, node_color=node_colors, cmap='Set2')
plt.title("Clasificación de nodos en el grafo CiteSeer")
plt.show()
